In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
# to convert strings to int for feature 
from pyspark.ml.feature import StringIndexer

In [4]:
spark = SparkSession.builder.appName('Crews_prediction').getOrCreate()

In [6]:
df = spark.read.csv('cruise_ship_info.csv' , inferSchema= True , header= True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [13]:
# show the count of group cruise_line 

df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [23]:
# Convert cruise line to int, so we can use it as feature 

indexer = StringIndexer(inputCol="Cruise_line", outputCol= "Cruise_line_group")

# fit the indexer and , save it in df. 
indexed = indexer.fit(df).transform(df) 

indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_group=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_group=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_line_group=1.0)]

In [24]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler

In [25]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_group']

In [26]:
assembler = VectorAssembler(inputCols=  ['Age',
                                        'Tonnage',
                                        'passengers',
                                        'length',
                                        'cabins',
                                        'passenger_density',
                                        'Cruise_line_group'] , 
                            outputCol= 'features')

In [27]:
output = assembler.transform(indexed)

In [29]:
# show the selected results 

output.select('features' , 'crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [30]:
# create a final data that has only features , crew 

final_data = output.select('features' , 'crew')

In [41]:
# train , test split 

train_data , test_data = final_data.randomSplit([0.7 , 0.3])

In [42]:
# compare train and test
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              106|
|   mean|8.020471698113218|
| stddev|3.389888994411096|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [43]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                52|
|   mean|7.3328846153846134|
| stddev|  3.71535992320722|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [44]:
# building regression modle 

ship_lr = LinearRegression(labelCol= 'crew') 

In [45]:
# fitting the model 
trained_ship_model = ship_lr.fit(train_data)

22/06/04 23:01:34 WARN Instrumentation: [75960d79] regParam is zero, which might cause numerical instability and overfitting.


In [46]:
# evaluating 
ship_result = trained_ship_model.evaluate(test_data)

In [47]:
# check errors 
print(ship_result.rootMeanSquaredError) 
print(ship_result.meanAbsoluteError) 
print(ship_result.meanSquaredError)

0.6355721034566432
0.5142502391589924
0.40395189869230186


In [48]:
# check r^2 
ship_result.r2

0.9701625958400598

In [51]:
# because the model preform very well , we need to do a reality check by checking the correlations 

from pyspark.sql.functions import corr 

df.select(corr('crew' , 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [52]:
# check another correlation 

df.select(corr('crew' , 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [53]:
# create unlabeled data just to check the model 

unalbeled_data = test_data.select('features') 

In [54]:
# show it 

unalbeled_data.show()

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,86.0,21.04,9...|
|[6.0,30.276999999...|
|[6.0,93.0,23.94,9...|
|[6.0,112.0,38.0,9...|
|[10.0,58.825,15.6...|
|[10.0,68.0,10.8,7...|
|[10.0,77.0,20.16,...|
|[10.0,90.09,25.01...|
|[10.0,110.0,29.74...|
|[10.0,138.0,31.14...|
|[11.0,85.0,18.48,...|
|[11.0,90.0,22.4,9...|
|[11.0,91.0,20.32,...|
|[11.0,138.0,31.14...|
|[12.0,2.329,0.94,...|
|[12.0,77.104,20.0...|
|[12.0,88.5,21.24,...|
|[12.0,90.09,25.01...|
|[12.0,91.0,20.32,...|
+--------------------+
only showing top 20 rows



In [56]:
predictions = trained_ship_model.transform(unalbeled_data)

In [57]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.197565272513938|
|[5.0,86.0,21.04,9...|  9.21598647162109|
|[6.0,30.276999999...|  4.24115547703099|
|[6.0,93.0,23.94,9...|10.538134996151996|
|[6.0,112.0,38.0,9...|11.386884156800402|
|[10.0,58.825,15.6...| 7.243934743219006|
|[10.0,68.0,10.8,7...| 6.599369129154569|
|[10.0,77.0,20.16,...| 8.783525705169607|
|[10.0,90.09,25.01...| 8.889379273335958|
|[10.0,110.0,29.74...| 12.15543220132244|
|[10.0,138.0,31.14...|13.102665237232193|
|[11.0,85.0,18.48,...| 8.775298849086608|
|[11.0,90.0,22.4,9...|10.056207984792831|
|[11.0,91.0,20.32,...| 9.243208789453004|
|[11.0,138.0,31.14...| 13.10064696363235|
|[12.0,2.329,0.94,...|0.4219726941310187|
|[12.0,77.104,20.0...| 8.788014879341487|
|[12.0,88.5,21.24,...|  9.48618860276768|
|[12.0,90.09,25.01...| 8.885342726136274|
|[12.0,91.0,20.32,...| 9.241190515853162|
+--------------------+------------